# 01.03.2025
## Theory

The Lovasz Theta number of an undirected graph G, which we will denote as $L(G)$ for simplicity, can be found by solving the following eigevalue problem:

$L(G) = min_{A \in S^{n}}\{\lambda_{max}(A): A_{i,j} = 1 \iff i=j \hspace{0.1 cm} $or$ \hspace{0.1 cm} (i,j) \not \in E\}$

where S^{n} denotes the set of $n$x$n$ real symmetric matrices.

An interesting property of $L(G)$ is given by Lovasz's Sandwich Theorem, namely:


$\omega(G) \leq L(\bar{G}) \leq \chi(G)$ where $\bar{G}$ is the complementary graph of $G$, $\omega(G)$ is the clique number and $\chi(G)$ is the chromatic number.

Thus, for perfect graphs, it is enough to find $L(\bar{G})$ with an accuracy of $\epsilon < \frac{1}{2}$ in order to know $\omega(G) = \alpha(\bar{G})$ and $\chi(G)$. This allows for efficient polynomial time algorithms for finding maximum independent sets and optimal colorings.  

It also satisfies 

$L(G \square H) = L(G) L(H)$ where $G \square H$ is the strong product of G and H.

Moroever, it upper bounds the Shannon Capacity $\Theta(G) \leq L(G)$.

## How the code works

### Encoding the graph
We will represent undirected graphs as vectors of vectors. 
graph_list will be a vector of length $n$ (number of vertices) and will have in entry $i$ the neighbors of vertex $i$ in an array.

Example:
graph_list = [[2,3],[1,3],[1,2]] means vertex $1$ is connected to $2,3$. Vertex $2$ is connected to $1,3$. Vertex $3$ is ocnnected to $1,2$. In this case graph_list encodes $C_{3}$.

##### function create_matrix(graph_list)
Creates the matrix $A_{i,j} = 1 \iff i=j \hspace{0.1 cm} $or$ \hspace{0.1 cm} (i,j) \not \in E$

##### function max_eigen(A)
Computes the maximum eigenvalue fo A via the power method

##### function randomize!(A)
Gives random uniform values in the interval $[-1,1]$ to the entries $A_{i,j} \hspace{0.1 cm} $s.t.$ \hspace{0.1 cm} (i,j) \not \in E$ and returns the minimum eigenvalue over all of them.

##### function Lovasz(g)
Calls the above functions in order and returns the minimum eigenvalue computed over all random matrices explored. Outputs the value of the eigenvalue and the matrix that achieves the minimum. 

#### function complement(g)
Returns the complementary graph list of a given graph list g.

### One problem

Inside the function $function randomize!(A)$, it is not clear how to select the interval where the random values should live in. I have not found many useful papers that might give some insight into the matter, so I have decided to somply try experimentally by progressively augmenting the interval length and then returning the lowest value among all chosen intervals. My intuiton tells me that the possitivive values of should not be much larger that 1, while the negative values can be somewhat larger in absoulte value (I suspect of the order of n/2).

The function $function randomize_flexible(A,l)$ allows to change the lower bound of the interval where the random variable lives.

In [78]:
using LinearAlgebra
using Distributions

In [129]:
#We will represent undirected graphs as vectors of vectors. 
# list will be a vector of length n (number of vertices) and will have in entry i the neighbors of vertex i in an array.
# Example:
# graph_list = [[2,3],[1,3],[1,2]] means vertex 1 is connected to 2,3. vertex 2 is connected to 1,3. vertex 3 is ocnnected to 1,2.

#------------------------CREATE THE MATRIX GIVEN THE LIST-------------------------------

function create_matrix(graph_list)
    A = ones(length(graph_list),length(graph_list))
    for i in 1:length(graph_list)
        if length(graph_list[i]) >= 1
            for j in 1:length(graph_list[i])
                A[i,graph_list[i][j]] = 0
            end
        end
    end
    return A
end 

#----------------------------COMPUTE LARGEST EIGENVALUE-------------------------------

function max_eigen(A)
    
    # Choose a random unitary vector q 
    # In: symmetric matrix A
    # Out: spectral radius of A
    
    q = rand(size(A)[2])
    q = q/norm(q,2)
    x = A*q
    λ = 0
    for k ∈ 1:10000
        q = x/norm(x)
        if abs(λ - dot(x,q)) < 10^-8 #enough precision has been reached
            λ = dot(x,q)
            break
        end
        λ = dot(x,q)
        x = A*q
    end
    return λ
end

#----------------------------MINIMIZE OVER SET OF SYMMETRIC MATRICES-------------------------------

function randomize!(A)
    dim = size(A)[1]
    for i in 1:dim
        for j in i+1:dim #We check only upper triangular matrix (without diagonal) since it will be symmetric
            if A[i,j] != 1.0
                r = rand(Uniform(-5,1)) #determining this ratio is still unknown to me a priori
                A[i,j] = r
                A[j,i] = r
            end
        end
    end
    return A
end

#----------------------------PUT IT ALL TOGETHER-------------------------------

function Lovasz(g)
    λ = 10^6 #Any sufficiently large starting constant
    matrix = []
    for k in 1:100000
        matrix = randomize!(create_matrix(g))
        candidate = max_eigen(matrix)
        if candidate < λ
            if λ - candidate < 10^-3
                λ = candidate
                break
            end
            λ = candidate
        end
    end
    print(matrix)
    return λ
end

#------------------------AUXILIARY FUNCTION TO CREATE GRAPH LIST OF COMPLEMENT GRAPH-------------------------------
function complement(g)
    g_complement = []
    g_complement_temporary = []
    for l in 1:length(g) 
        for k in 1:length(g) 
            if (k in g[l]) == false && (l != k)
                push!(g_complement_temporary,k)
            end
        end
        push!(g_complement, g_complement_temporary)
        g_complement_temporary = []
    end
    return g_complement
end

#----------------------------MINIMIZE OVER SET OF SYMMETRIC MATRICES-------------------------------

function randomize_flexible!(A,l)
    dim = size(A)[1]
    for i in 1:dim
        for j in i+1:dim #We check only upper triangular matrix (without diagonal) since it will be symmetric
            if A[i,j] != 1.0
                r = rand(Uniform(-l,1)) #determining this ratio is still unknown to me a priori
                A[i,j] = r
                A[j,i] = r
            end
        end
    end
    return A
end

#----------------------------PUT IT ALL TOGETHER WITH LOWER BOUND LIMITS-------------------------------

function Lovasz_lim(g,lim)
    λ = 10^6 #Any sufficiently large starting constant
    lowerbound = 1
    for l in 1:lim
        for k in 1:100000
            candidate = max_eigen(randomize_flexible!(create_matrix(g),l))
            if candidate < λ
                if λ - candidate < 10^-3 #Enough accuracy has been reached
                    λ = candidate
                    lowerbound = l
                    break
                end
                λ = candidate
                lowerbound = l
            end
        end
    end
    return println("λ = ", λ, " optimal lower bound = ", lowerbound)
end

Lovasz_lim (generic function with 1 method)

In [119]:
#---------------INITIALIZE THE GRAPH LIST---------------------------

#Path of length 6. Since it is perfect, we know that L(complement(p_6)) = 2
p_6 = Vector{Vector}(undef, 6)
for k in 2:length(p_6)-1
    p_6[k] = [k-1,k+1]
end
p_6[1] = [2]
p_6[length(p_6)] = [length(p_6)-1]


#Complete graph on 10 vertices. We know that L(G) = 1
k_10 = Vector{Vector}(undef, 10)
vec_temp = []
for k in 1:length(k_10)
    for j in 1:length(k_10)
        if j != k
            push!(vec_temp, j)
        end
    end
    k_10[k] = vec_temp
    vec_temp = []
end

#Cycle of length 5. We know that L(G) = √5 ≈ 2.23606797749979
g_5 = Vector{Vector}(undef, 5)
for k in 2:length(g_5)-1
    g_5[k] = [k-1,k+1]
end
g_5[1] = [2,length(g_5)]
g_5[length(g_5)] = [1,length(g_5)-1]

#Cycle of length 50. We know that L(G) = 25
g_50 = Vector{Vector}(undef, 50)
for k in 2:length(g_50)-1
    g_50[k] = [k-1,k+1]
end
g_50[1] = [2,length(g_50)]
g_50[length(g_50)] = [1,length(g_50)-1]

#Petersen graph. We know that L(G) = 4
g_pet = Vector{Vector}(undef, 10)
g_pet[1] = [2,5,6]
g_pet[2] = [1,3,7]
g_pet[3] = [2,4,8]
g_pet[4] = [3,5,9]
g_pet[5] = [1,4,10]
g_pet[6] = [1,8,9]
g_pet[7] = [2,9,10]
g_pet[8] = [3,6,10]
g_pet[9] = [4,6,7]
g_pet[10] = [5,7,8]


;

In [127]:
#lim = n/2 usually works fine 
#Examples:
#Lovasz(g_5)
#Lovasz_lim(g_pet,4)
#Lovasz_lim(complement(g_pet),4)
#Lovasz_lim(complement(k_10),20)
#Lovasz_lim(p_6,4)
#Lovasz_lim(complement(p_6),4)